In [26]:
import tensorflow as tf
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
import seaborn as sns
import os ,sys


In [27]:
f=open("report.txt","r")
h=f.readline()
lines=f.readlines()
print(lines)

['20,70,5.5,0\n', '25,80,5.2,1\n', '30,78,6.1,0\n', '55,64,5.7,1\n', '60,79,6.0,0\n', '75,60,5.5,1\n', '45,89,5.8,1\n', '60,80,5.6,0\n', '55,80,5.5,1\n', '60,68,5.7,0']


In [28]:
age=[]
wt=[]
ht=[]
db=[]
for line in lines:
    w=line.strip().split(',')
    a=int(w[0])
    wg=int(w[1])
    h=float(w[2])
    l=int(w[-1])
    age.append(a)
    wt.append(wg)
    ht.append(h)
    db.append(l)
print(age)
print(wt)
print(ht)
print(db)

[20, 25, 30, 55, 60, 75, 45, 60, 55, 60]
[70, 80, 78, 64, 79, 60, 89, 80, 80, 68]
[5.5, 5.2, 6.1, 5.7, 6.0, 5.5, 5.8, 5.6, 5.5, 5.7]
[0, 1, 0, 1, 0, 1, 1, 0, 1, 0]


In [29]:
x=np.c_[age,wt,ht]
y=np.c_[db]
print("x: input feature \n",x)
print()
print('-'*50)
print("y ::target \n",y)


x: input feature 
 [[20.  70.   5.5]
 [25.  80.   5.2]
 [30.  78.   6.1]
 [55.  64.   5.7]
 [60.  79.   6. ]
 [75.  60.   5.5]
 [45.  89.   5.8]
 [60.  80.   5.6]
 [55.  80.   5.5]
 [60.  68.   5.7]]

--------------------------------------------------
y ::target 
 [[0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]]


In [30]:
def std_dev(x):
    x=np.array(x)
    return ((((x-x.mean())**2).sum())/(x.size-1))**0.5

def scale(x):
    return (x-x.mean())/std_dev(x)

def sigmoid(x,w):
    z=x.dot(w)
    sig= 1/(1+np.exp(-z))
    return sig

def sigmoid_fun(x):   #  x= x.w
    return 1/(1+np.exp(-x))

def scale_matrix(x):
    ncol=x.shape[1]
    for i in range(ncol):
        x[:,i]=scale(x[:,i])
    return x    
        
def predict(x,w):  # w=[w1,w2,w3,w4]
    r=x
    for v in w:
        r=sigmoid_fun(r.dot(v))
    return r   

def derivative(x):
    return x*(1-x) 


In [31]:
# 3,5,7,5,1
# Random Initailization of weights
np.random.seed(100)
w1=2*np.random.random((3,5))-1
w2=2*np.random.random((5,7))-1
w3=2*np.random.random((7,5))-1
w4=2*np.random.random((5,1))-1
w=[w1,w2,w3,w4]

In [44]:
X=scale_matrix(x)
X

array([[-1.58701123, -0.53830365, -0.60764362],
       [-1.30858821,  0.58316229, -1.74697541],
       [-1.03016519,  0.3588691 ,  1.67101996],
       [ 0.36194993, -1.21118322,  0.15191091],
       [ 0.64037295,  0.4710157 ,  1.29124269],
       [ 1.47564202, -1.65976959, -0.60764362],
       [-0.19489612,  1.59248164,  0.53168817],
       [ 0.64037295,  0.58316229, -0.22786636],
       [ 0.36194993,  0.58316229, -0.60764362],
       [ 0.64037295, -0.76259684,  0.15191091]])

In [33]:
def train(x,y,w,iter,conv=0.0001):
    perr=0
    for i in range(iter):
        l1=sigmoid(x,w[0])
        l2=sigmoid(l1,w[1])
        l3=sigmoid(l2,w[2])
        l4=sigmoid(l3,w[3])
        cerr=((y-l4)**2).mean()
        diff=abs(perr-cerr)
        j=0
        if diff<=conv:
            print("Training is completed")
            j=1
            break
        if i%100==0:
            print(f"current error is ::{cerr} at {i+1} iteration") 
        e4=(y-l4)
        d4=e4*derivative(l4)
        e3=d4.dot(w[3].T)
        d3=e3*derivative(l3)
        e2=d3.dot(w[2].T)
        d2=e2*derivative(l2)
        e1=d2.dot(w[1].T)
        d1=e1*derivative(l1)

        w[0]+=x.T.dot(d1)
        w[1]+=l1.T.dot(d2)
        w[2]+=l2.T.dot(d3)
        w[3]+=l3.T.dot(d4)    
        
        perr=cerr
    if j==0:
        print('Training not completed successfully')
    return w    





In [34]:
beta=train(X,y,w,1000)


current error is ::0.2499556346536481 at 1 iteration
current error is ::0.17350417197515552 at 101 iteration
current error is ::0.15003452293393274 at 201 iteration
current error is ::0.04562875422976303 at 301 iteration
Training is completed


In [35]:
beta

[array([[-1.97780436,  2.19166763,  1.10289161,  1.42472296, -3.46535139],
        [-2.95300968,  2.09286216,  2.25397134, -1.65837914,  1.74408826],
        [ 3.07825154,  2.2530343 , -0.76894683,  0.34998355, -1.61054131]]),
 array([[ 3.98265789,  0.67465284, -1.32534868,  0.63865744,  0.11173327,
          0.13386656,  0.42961088],
        [ 1.40691729,  0.32482185, -0.86169712, -0.40589894, -3.60140116,
         -2.29369654,  0.85550534],
        [-1.14796138,  0.35616503,  1.27585953, -1.81369951, -2.31528613,
         -2.33898088,  0.91585458],
        [ 3.56651406, -1.40303461,  2.81925383, -2.3950441 ,  1.60228639,
          1.03148972, -0.85910384],
        [-3.38170097, -1.34378581,  0.89549158,  0.16692727, -1.69342946,
         -1.24778487,  0.45453077]]),
 array([[ 3.76208018, -0.22418747, -2.08805272,  1.45330213, -2.91558274],
        [ 0.37630885, -1.24365052, -1.11371174, -0.49304696, -0.68777283],
        [-2.07491592,  0.74553419,  1.1864003 , -1.58548262,  2.0869733

In [36]:
ycap=predict(X,beta)
ycap

array([[0.04845489],
       [0.98444334],
       [0.00284797],
       [0.89355151],
       [0.00689762],
       [0.95064926],
       [0.94290195],
       [0.08350869],
       [0.91928023],
       [0.11182575]])

In [37]:
ycap[ycap>=0.5]=1
ycap[ycap<0.5]=0
ycap

array([[0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.]])

In [38]:
np.c_[y,ycap]

array([[0., 0.],
       [1., 1.],
       [0., 0.],
       [1., 1.],
       [0., 0.],
       [1., 1.],
       [1., 1.],
       [0., 0.],
       [1., 1.],
       [0., 0.]])

In [39]:
def accuracy(y,ycap):
    r=y==ycap
    pcnt=r[r==True].size
    n=y.size
    return pcnt/n

accuracy(y,ycap)


1.0

In [40]:
l=[10,20,30,40,50,60]
l


[10, 20, 30, 40, 50, 60]

In [41]:
l.clear()


In [43]:
sigmoid_fun(0.04518)

0.5112930790821065

In [45]:
def tanh(x):
    num=np.exp(x)-np.exp(-x)
    den=np.exp(x)+np.exp(-x)
    return num/den

In [50]:
print(X.shape, w1.shape)
z=X.dot(w1)
z

(10, 3) (3, 5)


array([[ 2.85793373, -5.9738384 , -2.49637674, -1.58100506,  5.53933765],
       [-4.51158216, -5.58350764,  1.21453134, -3.44289249,  8.36538054],
       [ 6.12154102,  2.25814915, -1.61220537, -1.47801154,  1.50453707],
       [ 3.32838959, -1.3993051 , -2.44759212,  2.57744567, -3.61135292],
       [ 1.31732348,  5.29846967,  0.77502086,  0.58314514, -3.47722406],
       [ 0.11230454, -1.60859404, -1.64635425,  4.64224307, -7.02976765],
       [-2.68047738,  4.10360873,  2.96561874, -2.73252919,  2.5965063 ],
       [-3.69004627,  2.11057224,  2.19591016, -0.1344996 , -0.83504262],
       [-4.30842995,  0.64471031,  2.18086826, -0.66409107,  0.74143797],
       [ 1.45304341,  0.14973508, -1.12942087,  2.23019506, -3.79381229]])

In [52]:
tanh(2.85793373)

0.9934350353831833

In [51]:
tanh(z)

array([[ 0.99343504, -0.99998705, -0.9865176 , -0.91875871,  0.99996912],
       [-0.99975886, -0.99997173,  0.8380336 , -0.99795766,  0.99999989],
       [ 0.99999036,  0.97837751, -0.92348527, -0.90109477,  0.90596477],
       [ 0.99743274, -0.88520135, -0.98514608,  0.98852401, -0.99854141],
       [ 0.86611677,  0.99995002,  0.64983951,  0.52494763, -0.99809306],
       [ 0.11183477, -0.92295199, -0.92835564,  0.99981431, -0.99999843],
       [-0.99065107,  0.99945479,  0.99470367, -0.99157148,  0.98895089],
       [-0.99875369,  0.97106121,  0.97554634, -0.13369439, -0.68317437],
       [-0.99963801,  0.56809822,  0.97480891, -0.58107956,  0.63001316],
       [ 0.89629302,  0.14862598, -0.81082096,  0.97714841, -0.99898715]])